In [ ]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../creds/.env")

In [ ]:
from openai import AzureOpenAI

azure_endpoint = "https://cursor-gpt-4.openai.azure.com"
api_version="2024-02-15-preview"


cliient = AzureOpenAI(
        azure_endpoint=azure_endpoint,
        api_version=api_version,
    )

messages = [{"role": "system", "content": "You're an AI assistant designed to help users return current weather. When a user asks for the weather in a given location, you should call the get_current_weather function. Don't make assumptions about what values to use with functions. Ask for clarification if a user request is ambiguous. Only use the functions you have been provided with."}, {"role": "user", "content": "who is Jeff Dean?"} # What's the weather like in San Francisco, Tokyo, and Paris?
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        # mode details the better in desc
                        # "description": "The location of the hotel. The location should include the city and the state's abbreviation (i.e. Seattle, WA or Miami, FL)"
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

response = client.chat.completions.create(
    model="pjf-dpo-turbo-35",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, or none
    seed=42,
    temperature=0,
    max_tokens =200, 
    )

In [ ]:
# %pip install pinecone-client

from pinecone import Pinecone

pc = Pinecone(api_key='YOUR_API_KEY')
from pinecone import Pinecone, ServerlessSpec

pc.create_index(
    name="quickstart",
    dimension=8,
    metric="euclidean",
    spec=ServerlessSpec(
        cloud='aws', 
        region='us-east-1'
    ) 
) 

index = pc.Index("quickstart")
index.upsert(
    vectors=[
        {"id": "vec1", "values": [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]},
        {"id": "vec2", "values": [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]},
        {"id": "vec3", "values": [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]},
        {"id": "vec4", "values": [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]}
    ],
    namespace="ns1"
)

index.upsert(
    vectors=[
        {"id": "vec5", "values": [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]},
        {"id": "vec6", "values": [0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6]},
        {"id": "vec7", "values": [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7]},
        {"id": "vec8", "values": [0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]}
    ],
    namespace="ns2"
)

In [ ]:
index.describe_index_stats()

# Returns:
# {'dimension': 8,
#  'index_fullness': 0.0,
#  'namespaces': {'ns1': {'vector_count': 4}, 'ns2': {'vector_count': 4}},
#  'total_vector_count': 8}

index.query(
    namespace="ns1",
    vector=[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
    top_k=3,
    include_values=True
)

index.query(
    namespace="ns2",
    vector=[0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7],
    top_k=3,
    include_values=True
)

In [ ]:
# pc.delete_index("quickstart")

# export PINECONE_API_KEY=<YOUR_API_KEY>
# export INDEX_HOST=pdf-ai-do8rm7p.svc.apw5-4e34-81fa.pinecone.io

# curl -X POST "https://$INDEX_HOST/vectors/delete" \
#   -H "Api-Key: $PINECONE_API_KEY" \
#   -H 'Content-Type: application/json' \
#   -d '{
#     "deleteAll": true,
#     "namespace": "clugy1k2b0000xxk54xtc7bmy"
#   }
# '

In [ ]:
import os
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="ada_gcal",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

api_key = os.environ["AZURE_OPENAI_API_KEY"]
azure_endpoint = "https://cursor-gpt-4.openai.azure.com"
api_version="2024-02-15-preview"


llm = AzureOpenAI(
    model="pjf-dpo-turbo-35", # model = "deployment_name"
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

response = llm.complete("Paul Graham is ")
print(response)

from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model